In [1]:
"""
The process class stores each processes CPU and I/O bursts, process id, total burst time,
wait time, turnaround time, and response time.
"""
class Process:

    def __init__(self, pid, bursts):
        self.pid = pid
        self.bursts = bursts
        #Total burst time to calculate wait time
        self.total_burst_time = sum(bursts)
        self.wait_time = 0
        self.turnaround_time = 0
        #Set to 1 initially
        self.response_time = 1

P1 = Process(1, [5, 27, 3, 31, 5, 43, 4, 18, 6, 22, 4, 26, 3, 24, 4])
P2 = Process(2, [4, 48, 5, 44, 7, 42, 12, 37, 9, 76, 4, 41, 9, 31, 7, 43, 8])
P3 = Process(3, [8, 33, 12, 41, 18, 65, 14, 21, 4, 61, 15, 18, 14, 26, 5, 31, 6])
P4 = Process(4, [3, 35, 4, 41, 5, 45, 3, 51, 4, 61, 5, 54, 6, 82, 5, 77, 3])
P5 = Process(5, [16, 24, 17, 21, 5, 36, 16, 26, 7, 31, 13, 28, 11, 21, 6, 13, 3, 11, 4])
P6 = Process(6, [11, 22, 4, 8, 5, 10, 6, 12, 7, 14, 9, 18, 12, 24, 15, 30, 8])
P7 = Process(7, [14, 46, 17, 41, 11, 42, 15, 21, 4, 32, 7, 19, 16, 33, 10])
P8 = Process(8, [4, 14, 5, 33, 6, 51, 14, 73, 16, 87, 6])

processes = [P1, P2, P3, P4, P5, P6, P7, P8]

#Total for cpu bursts to later calculate cpu utilization
burst_time = 553


First Come First Serve (non-preemptive)

In [2]:

def fcfs(processes):
    clock = 0
    ready_queue = processes.copy()  #Initialzes a copy of all processes that start in the ready queue
    io_queue = []
    running_process = 0  #0 means no process is running
    running_time = 0

    def printExecution():
      print(f"Clock: {clock}")

      if running_process == 0:
          print("Running Process: None")
      else:
          print(f"Running Process: P{running_process.pid} Burst Time: {running_process.bursts[0]}")

      print("Ready Queue:", [f"P{p.pid}({p.bursts})" for p in ready_queue])
      print("I/O Queue:", [f"P{p[0].pid}({p[1]})" for p in io_queue])
      print("\n")

    #checks if there are processes in ready queue, io queue, or if a process is currently running
    while len(ready_queue) > 0 or len(io_queue) > 0 or running_process != 0:
        printExecution()

      #If a process is running, running time gets incremented
        if running_process != 0:
            running_time += 1
            #If the run time equals the burst value it removes the process
            if running_time == running_process.bursts[0]:
                running_process.bursts.pop(0)
                #if a process finishes its cpu burst and still has values, it gets sent to the io queue
                if running_process.bursts:
                    io_time = running_process.bursts.pop(0)
                    #adds the process and their io burst value to the io queue
                    io_queue.append((running_process, io_time))
                #else the process has finished
                else:
                    running_process.turnaround_time = clock
                    print(f"Process P{running_process.pid} has completed its total execution.")
                #resets the run time and running process
                running_process = 0
                running_time = 0

        #Checks if there are not processes currently running
        #Also checks if there are processes in the ready queue
        if running_process == 0 and len(ready_queue) > 0:
          #Sets the next process in the queue to run its cpu burst
            running_process = ready_queue.pop(0)
            running_time = 0

            if running_process.response_time == 1:
              #All processes are initialized with a response time of 1
              #This then sets the response time to the clock value if its their first time running
                running_process.response_time = clock

        #creates a new io queue
        new_io_queue = []
        #checks if a process still has time left for its io burst
        for p, time_left in io_queue:
            time_left -= 1
            #once a process has finished it is sent back to ready queue
            if time_left == 0:
                ready_queue.append(p)
            #else the remaining processes are added to a new queue to finish their io bursts
            else:
                new_io_queue.append((p, time_left))
        #remakes the io queue and increments the clock by 1
        io_queue = new_io_queue
        clock += 1

    #Calculates wait time for all processes
    for p in processes:
        p.wait_time = p.turnaround_time - p.total_burst_time
    #returns each process instance and the final run time
    return processes, clock


#Generates all the final results and prints them to the terminal
def generateStats(processData, clock):
  #prints all the final stats
    avg_wait_time = sum(p.wait_time for p in processes) / 8
    avg_turnaround_time = sum(p.turnaround_time for p in processes) / 8
    avg_response_time = sum(p.response_time for p in processes) / 8
    #burst_time is the sum of all the cpu bursts
    cpu_utilization = ( burst_time / clock)

    print("Process\tWaiting Time\tTurnaround Time\tResponse Time")
    for p in processData:
        print(f"P{p.pid}\t{p.wait_time}\t\t{p.turnaround_time}\t\t{p.response_time}")
    print(f"\nAverage Waiting Time: {avg_wait_time}")
    print(f"Average Turnaround Time: {avg_turnaround_time}")
    print(f"Average Response Time: {avg_response_time}")
    print(f"CPU Utilization: {cpu_utilization}%")
    print(f"Total Run Time: {clock}")

if __name__ == "__main__":
    processData, clock = fcfs(processes)
    generateStats(processData, clock)

Clock: 0
Running Process: None
Ready Queue: ['P1([5, 27, 3, 31, 5, 43, 4, 18, 6, 22, 4, 26, 3, 24, 4])', 'P2([4, 48, 5, 44, 7, 42, 12, 37, 9, 76, 4, 41, 9, 31, 7, 43, 8])', 'P3([8, 33, 12, 41, 18, 65, 14, 21, 4, 61, 15, 18, 14, 26, 5, 31, 6])', 'P4([3, 35, 4, 41, 5, 45, 3, 51, 4, 61, 5, 54, 6, 82, 5, 77, 3])', 'P5([16, 24, 17, 21, 5, 36, 16, 26, 7, 31, 13, 28, 11, 21, 6, 13, 3, 11, 4])', 'P6([11, 22, 4, 8, 5, 10, 6, 12, 7, 14, 9, 18, 12, 24, 15, 30, 8])', 'P7([14, 46, 17, 41, 11, 42, 15, 21, 4, 32, 7, 19, 16, 33, 10])', 'P8([4, 14, 5, 33, 6, 51, 14, 73, 16, 87, 6])']
I/O Queue: []


Clock: 1
Running Process: P1 Burst Time: 5
Ready Queue: ['P2([4, 48, 5, 44, 7, 42, 12, 37, 9, 76, 4, 41, 9, 31, 7, 43, 8])', 'P3([8, 33, 12, 41, 18, 65, 14, 21, 4, 61, 15, 18, 14, 26, 5, 31, 6])', 'P4([3, 35, 4, 41, 5, 45, 3, 51, 4, 61, 5, 54, 6, 82, 5, 77, 3])', 'P5([16, 24, 17, 21, 5, 36, 16, 26, 7, 31, 13, 28, 11, 21, 6, 13, 3, 11, 4])', 'P6([11, 22, 4, 8, 5, 10, 6, 12, 7, 14, 9, 18, 12, 24, 15, 30, 8])'

SJF (non-preemptive)

In [ ]:
"""
The process class stores each processes CPU and I/O bursts, process id, total burst time,
wait time, turnaround time, and response time.
"""
class Process:

    def __init__(self, pid, bursts):
        self.pid = pid
        self.bursts = bursts
        self.total_burst_time = sum(bursts) #Total burst time to calculate wait time
        self.wait_time = 0
        self.turnaround_time = 0
        self.response_time = 1 #Set to 1 initially

P1 = Process(1, [5, 27, 3, 31, 5, 43, 4, 18, 6, 22, 4, 26, 3, 24, 4])
P2 = Process(2, [4, 48, 5, 44, 7, 42, 12, 37, 9, 76, 4, 41, 9, 31, 7, 43, 8])
P3 = Process(3, [8, 33, 12, 41, 18, 65, 14, 21, 4, 61, 15, 18, 14, 26, 5, 31, 6])
P4 = Process(4, [3, 35, 4, 41, 5, 45, 3, 51, 4, 61, 5, 54, 6, 82, 5, 77, 3])
P5 = Process(5, [16, 24, 17, 21, 5, 36, 16, 26, 7, 31, 13, 28, 11, 21, 6, 13, 3, 11, 4])
P6 = Process(6, [11, 22, 4, 8, 5, 10, 6, 12, 7, 14, 9, 18, 12, 24, 15, 30, 8])
P7 = Process(7, [14, 46, 17, 41, 11, 42, 15, 21, 4, 32, 7, 19, 16, 33, 10])
P8 = Process(8, [4, 14, 5, 33, 6, 51, 14, 73, 16, 87, 6])

processes = [P1, P2, P3, P4, P5, P6, P7, P8]

#Total for cpu bursts to later calculate cpu utilization
burst_time = 553

In [ ]:
#Funtion to sort processes based on shortest burst times
def insert_sorted(readyQueue, process):
    i = 0
    while i < len(readyQueue) and readyQueue[i].bursts[0] <= process.bursts[0]:
        i += 1
    #Adds the sorted process to the ready queu
    readyQueue.insert(i, process)

def sjf(processes):
    clock = 0
    ready_queue = []
    io_queue = []
    running_process = 0 #0 means no process is running
    running_time = 0

    def printExecution():
      #Prints the current run time, the current running process, and the ready / IO queues
      print(f"Clock: {clock}")

      if running_process == 0:
          print("Running Process: None")
      else:
          print(f"Running Process: P{running_process.pid} Burst Time: {running_process.bursts[0]}")

      print("Ready Queue:", [f"P{p.pid}({p.bursts})" for p in ready_queue])
      print("I/O Queue:", [f"P{p[0].pid}({p[1]})" for p in io_queue])
      print("\n")


    #Sorts the processes and adds them to  the ready queue
    for p in processes:
        insert_sorted(ready_queue, p)

    #checks if there are processes in ready queue, io queue, or if a process is currently running
    while len(ready_queue) > 0 or len(io_queue) > 0 or running_process != 0:
        printExecution()
        #If a process is running, running time gets incremented
        if running_process:
            running_time += 1
            #If the run time equals the burst value it removes the process
            if running_time == running_process.bursts[0]:
                running_process.bursts.pop(0)
                #if a process finishes its cpu burst and still has bursts to run, it gets sent to the io queue
                if running_process.bursts:
                    #adds the io burst time
                    io_time = running_process.bursts.pop(0)
                    #adds the process to the io queue
                    io_queue.append((running_process, io_time))
                #else the process has finished
                else:
                    running_process.turnaround_time = clock
                    print(f"Process P{running_process.pid} has completed its total execution.")

                #resets the run time and running process
                running_process = 0
                running_time = 0

        #Checks if there are not processes currently running
        #Also checks if there are processes in the ready queue
        if running_process == 0 and len(ready_queue) > 0:
          #Sets the next process in the queue to run its cpu burst
          running_process = ready_queue.pop(0)

          if running_process.response_time == 1:
            #All processes are initialized with a response time of 1
            #This then sets the response time to the clock value if its their first time running
              running_process.response_time = clock

        #creates a new io queue
        new_io_queue = []
        #checks if a process still has time left for its io burst
        for p, time_left in io_queue:
            time_left -= 1
          #once a process has finished it is sorted by sjf and added to ready queue
            if time_left == 0:
                insert_sorted(ready_queue, p)
          #else the remaining processes are added to a new queue to finish their io bursts
            else:
                new_io_queue.append((p, time_left))
        #remakes the io queue and increments the clock by 1
        io_queue = new_io_queue
        clock += 1
    #Calculates wait time for all processes
    for p in processes:
        p.wait_time = p.turnaround_time - p.total_burst_time
    #returns each process instance and the final run time
    return processes, clock


def generateStats(processData, clock):
  #Prints all the final stats
    avg_wait_time = sum(p.wait_time for p in processes) / 8
    avg_turnaround_time = sum(p.turnaround_time for p in processes) / 8
    avg_response_time = sum(p.response_time for p in processes) / 8
    cpu_utilization = (burst_time / clock)

    print("Process\tWaiting Time\tTurnaround Time\tResponse Time")
    for p in processData:
        print(f"P{p.pid}\t{p.wait_time}\t\t{p.turnaround_time}\t\t{p.response_time}")
    print(f"\nAverage Waiting Time: {avg_wait_time}")
    print(f"Average Turnaround Time: {avg_turnaround_time}")
    print(f"Average Response Time: {avg_response_time}")
    print(f"CPU Utilization: {cpu_utilization}%")
    print(f"Total Run Time: {clock}")

if __name__ == "__main__":
    processData, clock = sjf(processes)
    generateStats(processData, clock)



Clock: 0
Running Process: None
Ready Queue: ['P4([3, 35, 4, 41, 5, 45, 3, 51, 4, 61, 5, 54, 6, 82, 5, 77, 3])', 'P2([4, 48, 5, 44, 7, 42, 12, 37, 9, 76, 4, 41, 9, 31, 7, 43, 8])', 'P8([4, 14, 5, 33, 6, 51, 14, 73, 16, 87, 6])', 'P1([5, 27, 3, 31, 5, 43, 4, 18, 6, 22, 4, 26, 3, 24, 4])', 'P3([8, 33, 12, 41, 18, 65, 14, 21, 4, 61, 15, 18, 14, 26, 5, 31, 6])', 'P6([11, 22, 4, 8, 5, 10, 6, 12, 7, 14, 9, 18, 12, 24, 15, 30, 8])', 'P7([14, 46, 17, 41, 11, 42, 15, 21, 4, 32, 7, 19, 16, 33, 10])', 'P5([16, 24, 17, 21, 5, 36, 16, 26, 7, 31, 13, 28, 11, 21, 6, 13, 3, 11, 4])']
I/O Queue: []


Clock: 1
Running Process: P4 Burst Time: 3
Ready Queue: ['P2([4, 48, 5, 44, 7, 42, 12, 37, 9, 76, 4, 41, 9, 31, 7, 43, 8])', 'P8([4, 14, 5, 33, 6, 51, 14, 73, 16, 87, 6])', 'P1([5, 27, 3, 31, 5, 43, 4, 18, 6, 22, 4, 26, 3, 24, 4])', 'P3([8, 33, 12, 41, 18, 65, 14, 21, 4, 61, 15, 18, 14, 26, 5, 31, 6])', 'P6([11, 22, 4, 8, 5, 10, 6, 12, 7, 14, 9, 18, 12, 24, 15, 30, 8])', 'P7([14, 46, 17, 41, 11, 42, 15, 21,

MLFQ

In [ ]:
"""
The process class stores each processes CPU and I/O bursts, process id, total burst time,
wait time, turnaround time, and response time.
"""
class Process:

    def __init__(self, pid, bursts):
        self.pid = pid
        self.bursts = bursts
        self.total_burst_time = sum(bursts) #Total burst time to calculate wait time
        self.wait_time = 0
        self.turnaround_time = 0
        self.response_time = 1 #Set to 1 initially

P1 = Process(1, [5, 27, 3, 31, 5, 43, 4, 18, 6, 22, 4, 26, 3, 24, 4])
P2 = Process(2, [4, 48, 5, 44, 7, 42, 12, 37, 9, 76, 4, 41, 9, 31, 7, 43, 8])
P3 = Process(3, [8, 33, 12, 41, 18, 65, 14, 21, 4, 61, 15, 18, 14, 26, 5, 31, 6])
P4 = Process(4, [3, 35, 4, 41, 5, 45, 3, 51, 4, 61, 5, 54, 6, 82, 5, 77, 3])
P5 = Process(5, [16, 24, 17, 21, 5, 36, 16, 26, 7, 31, 13, 28, 11, 21, 6, 13, 3, 11, 4])
P6 = Process(6, [11, 22, 4, 8, 5, 10, 6, 12, 7, 14, 9, 18, 12, 24, 15, 30, 8])
P7 = Process(7, [14, 46, 17, 41, 11, 42, 15, 21, 4, 32, 7, 19, 16, 33, 10])
P8 = Process(8, [4, 14, 5, 33, 6, 51, 14, 73, 16, 87, 6])

processes = [P1, P2, P3, P4, P5, P6, P7, P8]

#Total for cpu bursts to later calculate cpu utilization
burst_time = 553

In [ ]:
def mlfq(processes):
    clock = 0

    queue1 = processes.copy() #Initialzes a copy of all processes that start in RR queue 1
    queue2 = []
    queue3 = []
    io_queue = []

    running_process = 0 #0 means no process is running
    running_time = 0
    tq1 = 5
    tq2 = 10
    tq_count = 0 #used to keep track of tq time

    def printExecution():
      #Prints the current run time, the current running process, and the ready / IO queues
      print(f"Clock: {clock}")

      if running_process == 0:
          print("Running Process: None")
      else:
          print(f"Running Process: P{running_process.pid} Burst Time: {running_process.bursts[0]}")

      print("Queue 1:", [f"P{p.pid}({p.bursts})" for p in queue2])
      print("Queue 2:", [f"P{p.pid}({p.bursts})" for p in queue2])
      print("Queue 3:", [f"P{p.pid}({p.bursts})" for p in queue3])
      print("I/O Queue:", [f"P{p[0].pid}({p[1]})" for p in io_queue])
      print("\n")

    #checks if there are processes in ready queue, io queue, or if a process is currently running
    while len(queue1) > 0 or len(queue2) > 0 or len(queue3) > 0 or len(io_queue) > 0  or running_process !=  0:

        printExecution()

        #If no process is running it checks all the queues
        if running_process == 0:
            if queue1:
                running_process = queue1.pop(0)
                running_time = 0
                tq_count = tq1

            elif queue2:
                running_process = queue2.pop(0)
                running_time = 0
                tq_count = tq2

            elif queue3:
                running_process = queue3.pop(0)
                running_time = 0

        #Checsk if a process is running
        if running_process != 0:

            if running_process.response_time == 1:
              #All processes are initialized with a response time of 1
              #This then sets the response time to the clock value if its their first time running
                running_process.response_time = clock

            #Subtracts from the current processes cpu burst and increments run time
            running_process.bursts[0] -= 1
            running_time += 1
            #Checks if a burst is finished and removes  it
            if running_process.bursts[0] == 0:
                running_process.bursts.pop(0)
                #If a process has more burst after the cpu, it is added to the I/O queue
                if running_process.bursts:
                    io_time = running_process.bursts.pop(0)
                    io_queue.append((running_process, io_time))

                else:
                    #Else a process is finished, assigns turnaround time
                    running_process.turnaround_time = clock
                    print(f"Process P{running_process.pid} has completed its total execution.")
                #Resets running process and time
                running_process = 0
                running_time = 0
            #Checks if a run time reaches a tq count
            elif running_time == tq_count:
                #If a process in queue 1 exceeds tq(5), the process is added to RR queue 2
                if tq_count == tq1:
                    queue2.append(running_process)
                #If a process in queue 2 exceeds tq(10), the process is added to FCFS queue 3
                elif tq_count == tq2:
                    queue3.append(running_process)
                #Resets running process and time
                running_process = 0
                running_time = 0



        #creates a new io queue
        new_io_queue = []
        #checks if a process still has time left for its io burst
        for p, time_left in io_queue:
          #decreases the cpu bursts by 1
            time_left -= 1
            #Checks if a I/O burst is done
            if time_left == 0:
                #Removes the process and adds it back to queue 1
                queue1.append(p)
          #else the remaining processes are added to a new queue to finish their io bursts
            else:
                 new_io_queue.append((p, time_left))
        #remakes the io queue and increments the clock by 1
        io_queue = new_io_queue
        clock += 1
    #Calculates wait time for all processes
    for p in processes:
            p.wait_time = p.turnaround_time - p.total_burst_time

    #returns each process instance and the final run time
    return processes, clock

def generateStats(processData, clock):
    # Prints all the final stats
    avg_wait_time = sum(p.wait_time for p in processData) / 8
    avg_turnaround_time = sum(p.turnaround_time for p in processData) / 8
    avg_response_time = sum(p.response_time for p in processData) / 8
    cpu_utilization = (burst_time / clock)

    print("Process\tWaiting Time\tTurnaround Time\tResponse Time")
    for p in processData:
        print(f"P{p.pid}\t{p.wait_time}\t\t{p.turnaround_time}\t\t{p.response_time}")
    print(f"\nAverage Waiting Time: {avg_wait_time}")
    print(f"Average Turnaround Time: {avg_turnaround_time}")
    print(f"Average Response Time: {avg_response_time}")
    print(f"CPU Utilization: {cpu_utilization}%")
    print(f"Total Run Time: {clock}")

if __name__ == "__main__":

  processData, clock = mlfq(processes)
  generateStats(processData, clock)

Clock: 0
Running Process: None
Queue 1: []
Queue 2: []
Queue 3: []
I/O Queue: []


Clock: 1
Running Process: P1 Burst Time: 4
Queue 1: []
Queue 2: []
Queue 3: []
I/O Queue: []


Clock: 2
Running Process: P1 Burst Time: 3
Queue 1: []
Queue 2: []
Queue 3: []
I/O Queue: []


Clock: 3
Running Process: P1 Burst Time: 2
Queue 1: []
Queue 2: []
Queue 3: []
I/O Queue: []


Clock: 4
Running Process: P1 Burst Time: 1
Queue 1: []
Queue 2: []
Queue 3: []
I/O Queue: []


Clock: 5
Running Process: None
Queue 1: []
Queue 2: []
Queue 3: []
I/O Queue: ['P1(26)']


Clock: 6
Running Process: P2 Burst Time: 3
Queue 1: []
Queue 2: []
Queue 3: []
I/O Queue: ['P1(25)']


Clock: 7
Running Process: P2 Burst Time: 2
Queue 1: []
Queue 2: []
Queue 3: []
I/O Queue: ['P1(24)']


Clock: 8
Running Process: P2 Burst Time: 1
Queue 1: []
Queue 2: []
Queue 3: []
I/O Queue: ['P1(23)']


Clock: 9
Running Process: None
Queue 1: []
Queue 2: []
Queue 3: []
I/O Queue: ['P1(22)', 'P2(47)']


Clock: 10
Running Process: P3 Burst 